In [284]:
import matplotlib.pyplot as plt 
from torch import nn
import tensorflow as tf
import numpy as np
import torch
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader,TensorDataset 
from tqdm import tqdm
import torchsummary

### Cuda Check

In [12]:
USE_CUDA = torch.cuda.is_available()
device = torch.device('cuda:0' if USE_CUDA else 'cpu')

### getting ready for data

In [117]:
def get_one_hot(data, type_num):  # valid for any shape

        table = np.eye(type_num)
        def mapping(x): return table[x]

        _new = mapping(data)

        new_shape = [data.shape[i] for i in range(len(data.shape))]
        new_shape.append(type_num)
        _new = _new.reshape(new_shape)

        return _new

In [144]:
_random_state = 32

In [145]:
def get_MNIST_data(device):
    (x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()
    x_train, x_valid, y_train, y_valid = train_test_split(x_train, y_train, stratify=y_train, test_size=0.1, random_state=_random_state)
    
    y_train_encoded = get_one_hot(y_train, 10)
    y_valid_encoded = get_one_hot(y_valid, 10)
    y_test_encoded = get_one_hot(y_test, 10)
    
    x_train = torch.FloatTensor(x_train).to(device)
    y_train_encoded = torch.FloatTensor(y_train_encoded).to(device)

    x_valid = torch.FloatTensor(x_valid).to(device)
    y_valid_encoded = torch.FloatTensor(y_valid_encoded).to(device)

    x_test = torch.FloatTensor(x_test).to(device)
    y_test_encoded = torch.FloatTensor(y_test_encoded).to(device)

    x_train /= 255
    x_valid /= 255
    x_test /= 255
    
    x_train = x_train.unsqueeze(1)
    x_valid = x_valid.unsqueeze(1)
    x_test = x_test.unsqueeze(1)
    
    
    
    return x_train, y_train_encoded, x_valid, y_valid_encoded, x_test, y_test_encoded

In [146]:
x_train, y_train, x_valid, y_valid, x_test, y_test = get_MNIST_data(device)

### dataloader

In [147]:
class MNISTDataset(Dataset):
    
    
    def __init__(self,x,y):
        
        self.len = len(x)
        self.x_data = x
        self.y_data = y
        
        
    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]
        
        
    def __len__(self):
        return self.len
    

In [148]:
train_dataset = MNISTDataset(x_train, y_train)
train_loader = DataLoader(dataset = train_dataset, batch_size = 32, shuffle = True)

test_dataset = MNISTDataset(x_test, y_test)
test_loader = DataLoader(dataset = test_dataset, batch_size = 32, shuffle = True)

In [149]:
next(iter(train_loader))[0].shape

torch.Size([32, 1, 28, 28])

### model

In [150]:
#n_input = 784
#n_hidden = 128
#n_target = 10

#epochs = 50
#learning_rate = 1e-3

In [278]:
n_input = 784
n_hidden1 = 2500
n_hidden2 = 2000
n_hidden3 = 1500
n_hidden4 = 1000
n_hidden5 = 500
n_target = 10

epochs = 50
learning_rate = 5e-4

In [279]:
class MyModel(nn.Module):
    def __init__(self):
        super(MyModel, self).__init__()
        self.n_input = n_input
        self.n_hidden1 = n_hidden1
        self.n_hidden2 = n_hidden2
        self.n_hidden3 = n_hidden3
        self.n_hidden4 = n_hidden4
        self.n_hidden5 = n_hidden5
        self.n_target = n_target
        self._build()

    def _build(self):
        self.layer = nn.Sequential(
            nn.Linear(self.n_input, self.n_hidden1),
            nn.LeakyReLU(),
            
            nn.Linear(self.n_hidden1, self.n_hidden2),
            nn.LeakyReLU(),
            
            nn.Linear(self.n_hidden2, self.n_hidden3),
            nn.LeakyReLU(),
            
            nn.Linear(self.n_hidden3, self.n_hidden4),
            nn.LeakyReLU(),
            
            nn.Linear(self.n_hidden4, self.n_hidden5),
            nn.LeakyReLU(),
            
            nn.Linear(self.n_hidden5, self.n_target)
        )

    
    def forward(self, x):
        x = x.view(-1, self.n_input)
        return self.layer(x)

    def train(self,):
        self.criterion = nn.CrossEntropyLoss()
        self.optimizer = torch.optim.Adam(self.parameters(), lr=learning_rate)
        self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=int(epochs/10), gamma=0.6)
        
        
        
        self.dataloader = train_loader
        self.epochs = epochs
        
        global_step = 0
        for epoch in range(self.epochs):

            t = tqdm(self.dataloader)
            for i, data in enumerate(t):
                inputs, labels = data

                inputs = inputs.to(device)
                labels = labels.to(device)

                self.optimizer.zero_grad()
                self.zero_grad()

                y_pred = self.forward(inputs)
                loss = self.criterion(y_pred, labels)

                loss.backward()
                self.optimizer.step()
                
                global_step += 1
                t.set_description("epoch: {} | global_step: {:8d} | loss: {:.4f}".format(
                    epoch + 1, global_step, loss))
        
            predicts = self.forward(x_valid)
            predicts = torch.argmax(predicts, 1)
            label = torch.argmax(y_valid, 1)
            

            correct = 0
            for i in range(len(predicts)):
                if(predicts[i].item() == label[i]):
                    correct += 1

            acc = correct / len(predicts)
            print("epoch: {} | global_step: {} | valid acc: {:.3%}".format(
                epoch + 1, global_step, acc))
            
            
            self.scheduler.step()
            if(epoch % 5 == 0):
                print('lr:{}'.format(self.optimizer.param_groups[0]['lr']))
            
        
        
        
        
    def evaluate(self, x, y):

        pred = self.forward(x)
        pred = torch.argmax(pred, 1)

        label = y
        label = torch.argmax(label, 1)
        data_len = len(pred)
        cor = 0

        for i in range(data_len):
            if(pred[i] == label[i]):
                cor += 1

        acc = cor / data_len
        print("acc: {}".format(acc))

    def inference(self, x, y):
        pred = self.forward(x)
        pred = torch.argmax(pred, 1)

        print("inference: {}, label: {}".format(pred.item(), y.item()))


In [280]:
model = MyModel().to(device)

In [281]:
torchsummary.summary(model,(1,28,28,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                 [-1, 2500]       1,962,500
         LeakyReLU-2                 [-1, 2500]               0
            Linear-3                 [-1, 2000]       5,002,000
         LeakyReLU-4                 [-1, 2000]               0
            Linear-5                 [-1, 1500]       3,001,500
         LeakyReLU-6                 [-1, 1500]               0
            Linear-7                 [-1, 1000]       1,501,000
         LeakyReLU-8                 [-1, 1000]               0
            Linear-9                  [-1, 500]         500,500
        LeakyReLU-10                  [-1, 500]               0
           Linear-11                   [-1, 10]           5,010
Total params: 11,972,510
Trainable params: 11,972,510
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.00
Fo

### Train

In [282]:
model.train()

epoch: 1 | global_step:     1688 | loss: 0.1393: 100%|████████████████████████████| 1688/1688 [00:09<00:00, 171.76it/s]


epoch: 1 | global_step: 1688 | valid acc: 96.133%
lr:0.0005


epoch: 2 | global_step:     3376 | loss: 0.2642: 100%|████████████████████████████| 1688/1688 [00:08<00:00, 190.51it/s]


epoch: 2 | global_step: 3376 | valid acc: 96.733%


epoch: 3 | global_step:     5064 | loss: 0.1220: 100%|████████████████████████████| 1688/1688 [00:09<00:00, 185.13it/s]


...

epoch: 48 | global_step: 81024 | valid acc: 98.483%


epoch: 49 | global_step:    82712 | loss: -0.0000: 100%|██████████████████████████| 1688/1688 [00:08<00:00, 194.47it/s]


epoch: 49 | global_step: 82712 | valid acc: 98.483%


epoch: 50 | global_step:    84400 | loss: -0.0000: 100%|██████████████████████████| 1688/1688 [00:09<00:00, 187.28it/s]


epoch: 50 | global_step: 84400 | valid acc: 98.483%


### evaluation w/ test set
- 98.66%

In [283]:
model.evaluate(x_test, y_test)

acc: 0.9866
